# Carga de audios, transcripción, limpieza y almacenamiento

In [1]:
import os
#import mysql.connector
import re

# Install whisper package
#%pip install openai-whisper

import whisper
import subprocess

In [2]:
# Folderes donde se ubican los audios y donde se almacenan los txt
audio_folder = "C:/Users/Usuario/tfm_call_optimizer/audios/"
transcription_folder = "C:/Users/Usuario/tfm_call_optimizer/transcripciones"

In [3]:
# Load Whisper model
model = whisper.load_model("medium")

c:\Users\Usuario\anaconda3\envs\chatbot_rag\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location

In [4]:
def clean_transcription(text):
    """
    Limpia la transcripción eliminando muletillas, espacios extra
    y artefactos no deseados.
    """
    filler_words = ["eh", "mmm", "este", "o sea", "pues", "bueno", "entonces", "verdad", "sabes", "digo"]
    filler_pattern = r'\b(?:' + '|'.join(filler_words) + r')\b'

    text = re.sub(filler_pattern, '', text, flags=re.IGNORECASE)  # Elimina muletillas
    text = re.sub(r'\s+', ' ', text)  # Reemplaza múltiples espacios por uno solo
    text = re.sub(r'([.?!])\1+', r'\1', text)  # Elimina repeticiones innecesarias de signos de puntuación
    text = text.strip()  # Elimina espacios al inicio y al final

    return text


In [5]:
for filename in os.listdir(audio_folder):
    if filename.endswith(".wav") or filename.endswith(".mp3"):
        audio_path = os.path.join(audio_folder, filename)
        transcription_path = os.path.join(transcription_folder, filename.replace(".wav", ".txt").replace(".mp3", ".txt"))
        result = model.transcribe(audio_path)
        raw_text = result["text"]

        cleaned_text = clean_transcription(raw_text)
        with open(transcription_path, "w", encoding="utf-8") as f:
            f.write(cleaned_text)


c:\Users\Usuario\anaconda3\envs\chatbot_rag\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [4]:
# Ruta del archivo WAV
input_audio_path = "C:/Users/Usuario/tfm_call_optimizer/audios/MESAVOIP_20250114-163456_4007_6044038280-all.WAV"
output_audio_path = "C:/Users/Usuario/tfm_call_optimizer/audios/MESAVOIP_20250114-163456_4007_6044038280-all.mp3"

# Comando para convertir WAV a MP3
ffmpeg_command = [
    r"C:\Users\Usuario\ffmpeg-2025-03-06-git-696ea1c223-essentials_build\bin\ffmpeg.exe",
    "-i", input_audio_path,
    output_audio_path
]

# Ejecutar el comando
try:
    subprocess.run(ffmpeg_command, check=True)
    print(f"✅ El archivo se ha convertido exitosamente a: {output_audio_path}")
except subprocess.CalledProcessError as e:
    print(f"❌ Error en la conversión: {e}")

# Ahora, usa este archivo convertido para transcribirlo con whisper
audio_path = output_audio_path

✅ El archivo se ha convertido exitosamente a: C:/Users/Usuario/tfm_call_optimizer/audios/MESAVOIP_20250114-163456_4007_6044038280-all.mp3


In [7]:
filename = "MESAVOIP_20250114-163456_4007_6044038280-all.wav"  # Nombre del archivo de prueba
audio_path = os.path.join(audio_folder, filename)
transcription_path = os.path.join(transcription_folder, filename.replace(".wav", ".txt"))

# Transcribir audio
result = model.transcribe(audio_path)
raw_text = result["text"]

# Limpiar transcripción
#cleaned_text = clean_transcription(raw_text)

# Guardar en archivo de texto
with open(transcription_path, "w", encoding="utf-8") as f:
    f.write(cleaned_text)

print(f"✅ Saved cleaned transcription: {transcription_path}")


NameError: name 'cleaned_text' is not defined

In [6]:
# Transcribir el archivo de audio convertido
audio_path = "C:/Users/Usuario/tfm_call_optimizer/audios/MESAVOIP_20250114-163456_4007_6044038280-all.mp3"

# Transcribir audio
result = model.transcribe(audio_path)

# Obtener el texto transcrito
raw_text = result["text"]

# Imprimir el texto transcrito
print(f"Texto transcrito: {raw_text}")

# Guardar la transcripción en un archivo de texto
transcription_path = "C:/Users/Usuario/tfm_call_optimizer/transcripciones/MESAVOIP_20250114-163456_4007_6044038280-all.txt"
with open(transcription_path, "w", encoding="utf-8") as f:
    f.write(raw_text)

print(f"✅ Transcripción guardada en: {transcription_path}")

Texto transcrito:  Bienvenidos a la Fundación Universitaria de las Artes, un espacio para la creatividad. Si conoces el número de la expedición, pide esa información sobre nuestros cursos, diplomados y carreras profesionales, Marche 1. Para procesos de admisión, en registro, matrículas o certificados... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... Hola. Hola. Hola, caro. Te escucho, me escuchas. Sí, invito. Mira, tengo una línea en Facebook y tengo también una de WhatsApp. Exacto. Muchas gracias. Una de WhatsApp, la 317. ¿Ok? 699... 649. ¿6 qué? 649. 649. 0897. 0897. 0897. Listo. Sí, señora, está la línea en WhatsApp y la escribo en la 604. 604. 322. 322. 82, 83. 82, 83. Perfecto, Carolina. Muchas gracias, te lo agradezco. Muchísimas gracias, te lo agradezco. Muchísimas gracias, te lo agradezco. Igualmente, felicidades.
✅ Transcripción guardada en: C:/Users/Usuario/tfm_call_optimizer/transcripciones/MESAVOIP_20250114-163456_4007_604403

# Connect to MySQL
conn = mysql.connector.connect(
    host="your-mysql-server",
    user="your-username",
    password="your-password",
    database="your-database"
)
cursor = conn.cursor()

# Asegurémonos de que `ffmpeg` esté en el PATH antes de la transcripción
ffmpeg_path = r"C:\Users\Usuario\ffmpeg-2025-03-06-git-696ea1c223-essentials_build\bin\ffmpeg.exe"

# Verificar si ffmpeg está accesible desde Python
try:
    subprocess.run([ffmpeg_path, "-version"], check=True, capture_output=True)
    print("✅ ffmpeg está correctamente accesible.")
except FileNotFoundError:
    print("❌ No se pudo encontrar ffmpeg en la ruta especificada.")
    exit()


# Verificar si ffmpeg está accesible desde Python
try:
    subprocess.run([ffmpeg_path, "-version"], check=True, capture_output=True)
    print("✅ ffmpeg está correctamente accesible.")
except FileNotFoundError:
    print("❌ No se pudo encontrar ffmpeg en la ruta especificada.")
    exit()

# Asegurémonos de que `ffmpeg` esté en el PATH antes de la transcripción
ffmpeg_path = r"C:\Users\Usuario\ffmpeg-2025-03-06-git-696ea1c223-essentials_build\bin\ffmpeg.exe"

# Configurar ffmpeg manualmente en whisper
whisper.utils.ffmpeg_bin = ffmpeg_path  # Esto le dice a whisper dónde encontrar ffmpeg

In [ ]:
# Limpiar transcripción
cleaned_text = clean_transcription(raw_text)

# Guardar en archivo de texto
with open(transcription_path, "w", encoding="utf-8") as f:
    f.write(cleaned_text)

print(f"✅ Saved cleaned transcription: {transcription_path}")

NameError: name 'clean_transcription' is not defined


# Function to clean transcription text
def clean_transcription(text):
    """
    Cleans the transcription text by removing filler words, extra spaces,
    and unwanted artifacts.
    """
    # Define common filler words (can be extended)
    filler_words = ["eh", "mmm", "este", "o sea", "pues", "bueno", "entonces", "verdad", "sabes", "digo"]
    filler_pattern = r'\b(?:' + '|'.join(filler_words) + r')\b'

    # Remove filler words
    text = re.sub(filler_pattern, '', text, flags=re.IGNORECASE)

    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)

    # Remove unnecessary punctuation repetition (e.g., "Hello...." → "Hello.")
    text = re.sub(r'([.?!])\1+', r'\1', text)

    # Strip leading/trailing spaces
    text = text.strip()

    return text



for filename in os.listdir(audio_folder):
    if filename.endswith(".wav") or filename.endswith(".mp3"):
        audio_path = os.path.join(audio_folder, filename)
        transcription_path = os.path.join(transcription_folder, filename.replace(".wav", ".txt").replace(".mp3", ".txt"))

        # Transcribe audio
        result = model.transcribe(audio_path)
        raw_text = result["text"]

        # Clean transcription text
        cleaned_text = clean_transcription(raw_text)

        # Save cleaned transcription to a .txt file
        with open(transcription_path, "w", encoding="utf-8") as f:
            f.write(cleaned_text)

        # Insert cleaned transcription into MySQL
        sql = """
        INSERT INTO transcriptions (call_id, transcription, status)
        VALUES (%s, %s, %s)
        """
        values = (filename.replace(".wav", "").replace(".mp3", ""), cleaned_text, "Pending")

        cursor.execute(sql, values)
        conn.commit()

        print(f"✅ Saved cleaned transcription: {transcription_path} & stored in database.")

cursor.close()
conn.close()
